# Imports

In [1]:
import torch
import torch.nn as nn # all nn modules
import torch.optim as optim # optimization algorithms
import torch.nn.functional as F # activation functions like relu, tanh (all functions with no parameters)
from torch.utils.data import DataLoader # helps with daata
import torchvision.datasets as datasets # has many data sets
import torchvision.transforms as transforms

# Create a fully connected NN

In [2]:
# inherits from the nn module
# Our first linear layer take input_size, in this case 784 nodes to 50
# and our second linear layer takes 50 to the num_classes we have, in this case 10.
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    # create a forward function
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [3]:
# 28*28 images passes as 784
# 10 for no of digits
model = NN(784, 10)
# 64 = no of examples (images) mini batch size
x = torch.rand((64, 784))
model(x).shape

torch.Size([64, 10])

In [4]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Hyperparameters

In [5]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
epochs = 1

# Load dataset

In [6]:
train_data = datasets.MNIST(root = "data/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_data, batch_size = batch_size, shuffle=True)

A:\Anacond\envs\deeplearning\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
test_data = datasets.MNIST(root = "data/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_data, batch_size = batch_size, shuffle=True)

# Initialize the network

In [8]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

# Loss & optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the network

In [10]:
for epoch in range(epochs):
    for batch_index, (data, targets) in enumerate(train_loader):
        
        # get data to cuda if possible
        
        data = data.to(device = device)
        targets = targets.to(device = device)
        
        # data has shape (64, 1, 28, 28)
        # 64 training examples
        # 1 as we are using gray scale images
        # 28 * 28 height, width
        
        # we need to reshape this to (65, 784)
        
        data = data.reshape(data.shape[0], -1)
        
        # forward
        
        score = model(data)
        loss = criterion(score, targets)
        
        # backward
        # set gradients to zero for each batch, so it does not store back prop calculation from previous forward props
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent
        optimizer.step()

# check accuracy of model

In [11]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("checking accuracy on testing data")

    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


        print(f"Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
            

In [12]:
check_accuracy(train_loader, model)

checking accuracy on training data
Got 55870/60000 with accuracy 93.12


In [13]:
check_accuracy(test_loader, model)

checking accuracy on testing data
Got 9301/10000 with accuracy 93.01
